# Dialogue and Narrative Coursework - Subtask 1 - Cosine Similarity

In [1]:
## imports
# %%capture
# !pip install datasets

from datasets import load_dataset, Dataset
import pandas as pd

In [2]:
# # Link Google Drive to get new dataset
# from google.colab import drive
# drive.mount('/gdrive')

# # %cd drive/MyDrive/Colab\ Notebooks/Dialogue\ &\ Narrative
# %cd ~
# %cd /gdrive/My\ Drive
# %cd Colab\ Notebooks/Dialogue\ &\ Narrative
# print()

# %ls .

In [3]:
new_val_data = pd.read_csv('doc2dial_rc_val.csv')
new_val_data.head()

,Unnamed: 0,id,question,context,answers,spans,domain,title
0,0,dea7174409afbfe0af0ace21e7f318ae_1,user:My insurance ended so what should i do,Many DMV customers make easily avoidable mista...,{'text': ['Because we all pay indirectly for c...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
1,1,dea7174409afbfe0af0ace21e7f318ae_3,user:Don't do that I'll get insurance \tagent:...,Many DMV customers make easily avoidable mista...,{'text': ['we mail you an insurance inquiry le...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
2,2,dea7174409afbfe0af0ace21e7f318ae_5,"user:I have, that is why I am here to clear th...",Many DMV customers make easily avoidable mista...,{'text': ['Learn more about how to change the ...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
3,3,dea7174409afbfe0af0ace21e7f318ae_7,user:Will I still have to turn in anything or ...,Many DMV customers make easily avoidable mista...,{'text': ['you won t learn that you have an in...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
4,4,dea7174409afbfe0af0ace21e7f318ae_9,"user:OKay, well I just logged in and I have in...",Many DMV customers make easily avoidable mista...,{'text': ['Because we all pay indirectly for c...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0


In [4]:
new_val_data["question"].iloc[12]

"user:What if I move to another state? \tagent:Your license will be suspended. \tuser:What if I don't change it? \tagent:You must change it immediately. \tuser:What if I forgot to update my address? \tagent:Your license and registration could be suspended. \tuser:What if my insurance had lapsed? \tuser:Yes, I just have to change the address. \tagent:Is your license current? \tagent:You need to report it to the DMV. \tuser:I need to change my address."

In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import ast


[nltk_data] Downloading package punkt to /home/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def train_predict_doc2vec(dataset, index):
  instance = dataset.iloc[index]
  docs = ast.literal_eval(instance['spans'])

  tokenized_docs = {}
  for i, d in docs.items():
      tokenized_docs[int(i)] = word_tokenize(d.lower())

  tagged_data = []
  for i, d in tokenized_docs.items():
    tagged_data.append(TaggedDocument(d, [i]))

  model = Doc2Vec(tagged_data, vector_size=80, window=5, min_count=1, workers=-1, epochs = 100)

  test_doc = instance['question'][5:]

  predictions = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)], topn=30)
  # print(predictions)

  ids = []
  count = 0
  sum_score = 0
  pred = 0
  tol_zone = 5
  while True:

    id, score = predictions[pred]
    ids.append(id)
    count += 1
    sum_score += score

    pred += 1
    next_id, next_score = predictions[pred]

    possible_next_ids = list(range(id-tol_zone, id+tol_zone+1))
    # print(possible_next_ids)
    if next_id not in possible_next_ids:
      break

  sorted_ids = sorted(ids)
  text = ''
  for i in sorted_ids:
    text += docs.get(str(i), '')

  final_score = sum_score / count
  return {"id":instance["id"], "prediction_text":text, "no_answer_probability":1.0-final_score}

In [7]:
## Example
new_val_data_slice = new_val_data.sample(10, random_state=2)

for i in range(len(new_val_data_slice)):
  print("QUESTION:", new_val_data_slice["question"].iloc[i][5:])
  print("ANSWER:  ", train_predict_doc2vec(new_val_data_slice, i)["prediction_text"])
  print()

QUESTION: hello I need information about the issue of driver's licenses and Drivers from other countries
ANSWER:   If you plan to return to your home country and will need your foreign driver license , you must have US Department of Homeland Security DHS documentation that supports your legal status in this country. 

QUESTION: What if the temporary visitor status is extended? 	agent:do not However, you must have DHS documentation from the US Department of Homeland Security. USA That supports its legal status in this country. The expiration date on your DHS document is added to your New York State non-driver's permit, driver's license, or ID card. 	user:you must be a US citizen. USA To get a New York driver's license? 	agent:Learn how to change your out-of-state driver's license 	user:How can I change my out-of-state driver's license? 	agent:You must present an International Driving Permit or a certified translation of your driving license to the test drive. The translation must be cer

In [8]:
from tqdm import tqdm

preds = []
# new_val_data_slice = new_val_data[:100]
new_val_data_slice = new_val_data[:]

for i in tqdm(range(len(new_val_data_slice))):
  preds.append(train_predict_doc2vec(new_val_data_slice, i))

preds

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 722/722 [00:53<00:00, 13.56it/s]


[{'id': 'dea7174409afbfe0af0ace21e7f318ae_1',
  'prediction_text': 'Because we all pay indirectly for crashes involving uninsured motorists , DMV works with insurance companies to electronically monitor your insurance coverage , ',
  'no_answer_probability': 0.6926852762699127},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_3',
  'prediction_text': 'It has all the answers you need to efficiently perform any DMV transaction. ',
  'no_answer_probability': 0.7144370079040527},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_5',
  'prediction_text': 'Learn more about About the NYS Driver Point System [4] and how to Pay Driver Responsibility Assessment [5 ] ',
  'no_answer_probability': 0.7412656545639038},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_7',
  'prediction_text': 'you must report a change of address to DMV within ten days of moving. ',
  'no_answer_probability': 0.7254763245582581},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_9',
  'prediction_text': 'Insurance policies must be from a company

In [9]:
import json

file = 'predictions_subtask1_cosine_simple.json'
with open(file, 'w') as outfile:
    json.dump(preds, outfile)
    
    
import os
cmd = 'python sharedtask_utils.py --task subtask1 --prediction_json '+file
os.system(cmd)

Reusing dataset doc2dial (/home/matt/.cache/huggingface/datasets/doc2dial/doc2dial_rc/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
2022-01-07 10:56:06.717666: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-07 10:56:06.717687: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


predictions_subtask1_cosine_simple.json
{'exact': 0.4155124653739612, 'f1': 11.33066904788522, 'total': 722, 'HasAns_exact': 0.4155124653739612, 'HasAns_f1': 11.33066904788522, 'HasAns_total': 722, 'best_exact': 0.4155124653739612, 'best_exact_thresh': 0.7799205183982849, 'best_f1': 11.330669047885213, 'best_f1_thresh': 0.8813425302505493}


0